In [1]:
import pandas as pd
import numpy as np
import time
from pykrx import stock
import FinanceDataReader as fdr
import warnings

warnings.filterwarnings(action='ignore')

In [31]:
import os
os.getcwd()

'C:\\Users\\PC\\OneDrive\\Data Science\\Weak-supervision\\생존편향 제거'

In [2]:
df_ = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\remove_survival_bias 데이터\new_df_bfconv.csv', encoding='euc-kr').set_index('Symbol Name')
df_

,Symbol,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Symbol Name,,,,,,,,,,,,,,,,,,,,,
조흥은행,A000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(1신),A000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(2신),A000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
동화약품,A000020,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
동화약품공업(1신),A000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
잉글우드랩,A950140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
코오롱티슈진,A950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
JTC,A950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [4]:
# convert symbol to eliminate useless 'A' (ex. A000010 -> 000010)
def symbol_convertor(str_):
    return str_[1:]

# 전체 기업코드 변환
symbol_origin = list(df_.loc[:'SNK','Symbol'].values)
symbol_origin = list(map(symbol_convertor, symbol_origin))

In [5]:
symbol_name_origin = list(df_.index.values)[:-1]   # 전체 기업이름(마지막 '편입개수'행만 빼고)

In [15]:
# make dictionary for corporate name & corporate code
symbol_dic = {}
for i in range(len(symbol_origin)):
    name = symbol_name_origin[i]
    code = symbol_origin[i]
    symbol_dic[name] = code
list(symbol_dic.items())[:5]

[('조흥은행', '000010'),
 ('조흥은행(1신)', '000011'),
 ('조흥은행(2신)', '000012'),
 ('동화약품', '000020'),
 ('동화약품공업(1신)', '000021')]

In [16]:
date_df = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\date_df.csv', encoding='euc-kr').set_index('Date')
date_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
Date,,,,,,,,,,,,,,,,,,,,,
20100101,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100102,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100103,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100104,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100105,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191228,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191229,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O


In [17]:
# rename한 columns : KOSPI200 기업이름
lv2_companies_rename = list(date_df.columns.values)
lv2_companies_rename

['AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ제일제당',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'GKL',
 'GS',
 'GS건설',
 'GS리테일',
 'HDC',
 'HDC현대산업개발',
 'HSD엔진',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 '케이씨씨',
 'KC그린홀딩스',
 'KISCO홀딩스',
 'KPX케미칼',
 '케이티',
 '케이티앤지',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생명과학',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS산전',
 'NAVER',
 'NH투자증권',
 'OCI',
 '포스코',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S-Oil',
 'SBS',
 'SK',
 'SKC',
 'SK네트웍스',
 'SK디스커버리',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'SK하이닉스',
 'SPC삼립',
 'STX',
 'STX엔진',
 'STX조선해양',
 'STX중공업',
 '티비에이치글로벌',
 '강원랜드',
 '경방',
 '고려아연',
 '고려제강',
 '광동제약',
 '광전자',
 '국도화학',
 '금호산업',
 '금호석유화학',
 '금호전기',
 '금호타이어',
 '기아자동차',
 '기업은행',
 '남양유업',
 '남해화학',
 '넥센타이어',
 '넷마블',
 '녹십자',
 '녹십자홀딩스',
 '농심',
 '대교',
 '대구은행',
 '대덕GDS',
 '대덕전자',
 '대림산업',
 '대상',
 '대우건설',
 '대우조선해양',
 '대웅제약',
 '대원강업',
 '대한유화',
 '대한전선',
 '대한제강',
 '대한제당',
 '대한제분',
 '대한항공',
 '도레이케미칼',
 '동국제강',
 '동부제철',
 '동서',
 '동

In [18]:
len(lv2_companies_rename)

310

In [19]:
daily_k200 = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\daily_k200.csv', encoding='euc-kr').set_index('Unnamed: 0')
daily_k200

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주


In [20]:
daily_k200.fillna('')

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주


In [21]:
# lv2_index : 20100101 ~ 20191231 전체 기간이 담겨있는 Date list
lv2_index = list(date_df.index.values)
lv2_index = list(map(str, lv2_index))
lv2_index

['20100101',
 '20100102',
 '20100103',
 '20100104',
 '20100105',
 '20100106',
 '20100107',
 '20100108',
 '20100109',
 '20100110',
 '20100111',
 '20100112',
 '20100113',
 '20100114',
 '20100115',
 '20100116',
 '20100117',
 '20100118',
 '20100119',
 '20100120',
 '20100121',
 '20100122',
 '20100123',
 '20100124',
 '20100125',
 '20100126',
 '20100127',
 '20100128',
 '20100129',
 '20100130',
 '20100131',
 '20100201',
 '20100202',
 '20100203',
 '20100204',
 '20100205',
 '20100206',
 '20100207',
 '20100208',
 '20100209',
 '20100210',
 '20100211',
 '20100212',
 '20100213',
 '20100214',
 '20100215',
 '20100216',
 '20100217',
 '20100218',
 '20100219',
 '20100220',
 '20100221',
 '20100222',
 '20100223',
 '20100224',
 '20100225',
 '20100226',
 '20100227',
 '20100228',
 '20100301',
 '20100302',
 '20100303',
 '20100304',
 '20100305',
 '20100306',
 '20100307',
 '20100308',
 '20100309',
 '20100310',
 '20100311',
 '20100312',
 '20100313',
 '20100314',
 '20100315',
 '20100316',
 '20100317',
 '20100318',

In [22]:
# all_list에 20100101 ~ 20191231 기간동안의 모든 KOSPI200 리스트를 담음 (202 * 3,652 = 737,704)
all_list = []
for d in lv2_index:
    all_list = all_list + list(daily_k200.loc[:, d])

In [23]:
lv2_companies = list(set(all_list))
lv2_companies   # 312개

[nan,
 '현대증권',
 '오뚜기',
 '동양',
 'CJ',
 '쌍용양회',
 '현대모비스',
 '일진머티리얼즈',
 '삼성전자',
 '하이트맥주',
 '대덕GDS',
 '한온시스템',
 '대우조선해양',
 '우리은행',
 '우리금융',
 '코스맥스',
 '한화솔루션',
 '외환은행',
 '두산중공업',
 '크라운해태홀딩스',
 '동아쏘시오홀딩스',
 '미래에셋대우',
 '국도화학',
 '만도',
 '무학',
 '신세계',
 '현대로템',
 '퍼시스',
 '서흥',
 '광동제약',
 '디와이',
 '율촌화학',
 '한국타이어앤테크놀로지',
 'SBS',
 'LG유플러스',
 '부광약품',
 '카프로',
 '삼성화재',
 '신성이엔지',
 '삼성카드',
 '현대상선',
 '알앤엘바이오',
 '제일모직',
 '서연',
 '일신방직',
 '현대제철',
 '한솔케미칼',
 '도레이케미칼',
 '세하',
 '한국금융지주',
 '동부제철',
 '보령제약',
 'LG',
 '삼성전기',
 '디티알오토모티브',
 'STX조선해양',
 '코스맥스비티아이',
 '현대홈쇼핑',
 '동화약품',
 'SK',
 '동원F&B',
 '쌍용차',
 '삼성바이오로직스',
 '신세계인터내셔날',
 '한화화인케미칼',
 '코리아써키트',
 '종근당',
 '대상',
 '대구은행',
 '종근당홀딩스',
 '사조동아원',
 'HDC현대산업개발',
 'NH투자증권',
 '금호전기',
 '롯데쇼핑',
 'HSD엔진',
 '일동홀딩스',
 '삼성중공업',
 '대한제분',
 'NAVER',
 'BGF',
 'LG이노텍',
 '녹십자',
 '메리츠종금증권',
 '포스코플랜텍',
 'DB손해보험',
 '롯데칠성',
 '삼성증권',
 '신도리코',
 '코오롱인더',
 'GS',
 '셀트리온',
 'KT',
 '삼성SDI',
 '일진디스플',
 '한진중공업',
 '현대글로비스',
 '삼진제약',
 'LS',
 '한국철강',
 '한국단자',
 'BGF리테일',
 'CJ CGV',
 '락앤락',
 '케이씨오에너

In [24]:
del lv2_companies[0]
lv2_companies = sorted(lv2_companies)   # 순서대로 정렬
lv2_companies   # 311개

['AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ제일제당',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'GKL',
 'GS',
 'GS건설',
 'GS리테일',
 'HDC',
 'HDC현대산업개발',
 'HSD엔진',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KC그린홀딩스',
 'KISCO홀딩스',
 'KPX케미칼',
 'KT',
 'KT&G',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생명과학',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS산전',
 'NAVER',
 'NH투자증권',
 'OCI',
 'POSCO',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S-Oil',
 'SBS',
 'SK',
 'SKC',
 'SK네트웍스',
 'SK디스커버리',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'SK하이닉스',
 'SPC삼립',
 'STX',
 'STX엔진',
 'STX조선해양',
 'STX중공업',
 'TBH글로벌',
 '강원랜드',
 '경방',
 '고려아연',
 '고려제강',
 '광동제약',
 '광전자',
 '국도화학',
 '금호산업',
 '금호석유',
 '금호전기',
 '금호타이어',
 '기아차',
 '기업은행',
 '남양유업',
 '남해화학',
 '넥센타이어',
 '넷마블',
 '녹십자',
 '녹십자홀딩스',
 '농심',
 '대교',
 '대구은행',
 '대덕GDS',
 '대덕전자',
 '대림산업',
 '대상',
 '대우건설',
 '대우조선해양',
 '대웅제약',
 '대원강업',
 '대한유화',
 '대한전선',
 '대한제강',
 '대한제당',
 '대한제분',
 '대한항공',
 '도레이케미칼',
 '동국제강',
 '동부제철',
 '동서',
 '동아쏘시오홀딩스

In [25]:
lv2_companies.index('우리금융지주')

194

In [26]:
del lv2_companies[194]

In [27]:
# Data Guide 상 기업이름(수정하기 전) → symbol_dic이용할 때 써먹을 것
lv2_companies

['AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ제일제당',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'GKL',
 'GS',
 'GS건설',
 'GS리테일',
 'HDC',
 'HDC현대산업개발',
 'HSD엔진',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KC그린홀딩스',
 'KISCO홀딩스',
 'KPX케미칼',
 'KT',
 'KT&G',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생명과학',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS산전',
 'NAVER',
 'NH투자증권',
 'OCI',
 'POSCO',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S-Oil',
 'SBS',
 'SK',
 'SKC',
 'SK네트웍스',
 'SK디스커버리',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'SK하이닉스',
 'SPC삼립',
 'STX',
 'STX엔진',
 'STX조선해양',
 'STX중공업',
 'TBH글로벌',
 '강원랜드',
 '경방',
 '고려아연',
 '고려제강',
 '광동제약',
 '광전자',
 '국도화학',
 '금호산업',
 '금호석유',
 '금호전기',
 '금호타이어',
 '기아차',
 '기업은행',
 '남양유업',
 '남해화학',
 '넥센타이어',
 '넷마블',
 '녹십자',
 '녹십자홀딩스',
 '농심',
 '대교',
 '대구은행',
 '대덕GDS',
 '대덕전자',
 '대림산업',
 '대상',
 '대우건설',
 '대우조선해양',
 '대웅제약',
 '대원강업',
 '대한유화',
 '대한전선',
 '대한제강',
 '대한제당',
 '대한제분',
 '대한항공',
 '도레이케미칼',
 '동국제강',
 '동부제철',
 '동서',
 '동아쏘시오홀딩스

In [28]:
len(lv2_companies)

310

In [30]:
for i in range(310):
    if lv2_companies[i] != lv2_companies_rename[i]:
        print(lv2_companies[i], lv2_companies_rename[i])

KCC 케이씨씨
KT 케이티
KT&G 케이티앤지
POSCO 포스코
TBH글로벌 티비에이치글로벌
금호석유 금호석유화학
기아차 기아자동차
롯데칠성 롯데칠성음료
삼성화재 삼성화재해상보험
삼영전자 삼영전자공업
쌍용양회 쌍용양회공업
쌍용차 쌍용자동차
아모레G 아모레퍼시픽그룹
알앤엘바이오 알바이오
외환은행 하나은행
우리금융 우리금융지주
전기초자 한국전기초자
케이씨오에너지 아진카인텍
한국단자 한국단자공업
한국전력 한국전력공사
현대엘리베이 현대엘리베이터
현대증권 케이비증권
현대차 현대자동차


In [21]:
samsung = fdr.DataReader('005930', '2010-01-01', '2019-12-31')
lv2_index_rename = list(samsung.index)
lv2_index_rename

[Timestamp('2010-01-04 00:00:00'),
 Timestamp('2010-01-05 00:00:00'),
 Timestamp('2010-01-06 00:00:00'),
 Timestamp('2010-01-07 00:00:00'),
 Timestamp('2010-01-08 00:00:00'),
 Timestamp('2010-01-11 00:00:00'),
 Timestamp('2010-01-12 00:00:00'),
 Timestamp('2010-01-13 00:00:00'),
 Timestamp('2010-01-14 00:00:00'),
 Timestamp('2010-01-15 00:00:00'),
 Timestamp('2010-01-18 00:00:00'),
 Timestamp('2010-01-19 00:00:00'),
 Timestamp('2010-01-20 00:00:00'),
 Timestamp('2010-01-21 00:00:00'),
 Timestamp('2010-01-22 00:00:00'),
 Timestamp('2010-01-25 00:00:00'),
 Timestamp('2010-01-26 00:00:00'),
 Timestamp('2010-01-27 00:00:00'),
 Timestamp('2010-01-28 00:00:00'),
 Timestamp('2010-01-29 00:00:00'),
 Timestamp('2010-02-01 00:00:00'),
 Timestamp('2010-02-02 00:00:00'),
 Timestamp('2010-02-03 00:00:00'),
 Timestamp('2010-02-04 00:00:00'),
 Timestamp('2010-02-05 00:00:00'),
 Timestamp('2010-02-08 00:00:00'),
 Timestamp('2010-02-09 00:00:00'),
 Timestamp('2010-02-10 00:00:00'),
 Timestamp('2010-02-

In [22]:
def length_adjust(str_) : 
    return (2-len(str_)) * '0' + str_

def date_convertor(date) : 
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd

In [23]:
lv2_index_rename = list(map(date_convertor,lv2_index_rename))

In [24]:
lv2_index_rename = list(map(str, lv2_index_rename))
lv2_index_rename

['20100104',
 '20100105',
 '20100106',
 '20100107',
 '20100108',
 '20100111',
 '20100112',
 '20100113',
 '20100114',
 '20100115',
 '20100118',
 '20100119',
 '20100120',
 '20100121',
 '20100122',
 '20100125',
 '20100126',
 '20100127',
 '20100128',
 '20100129',
 '20100201',
 '20100202',
 '20100203',
 '20100204',
 '20100205',
 '20100208',
 '20100209',
 '20100210',
 '20100211',
 '20100212',
 '20100216',
 '20100217',
 '20100218',
 '20100219',
 '20100222',
 '20100223',
 '20100224',
 '20100225',
 '20100226',
 '20100302',
 '20100303',
 '20100304',
 '20100305',
 '20100308',
 '20100309',
 '20100310',
 '20100311',
 '20100312',
 '20100315',
 '20100316',
 '20100317',
 '20100318',
 '20100319',
 '20100322',
 '20100323',
 '20100324',
 '20100325',
 '20100326',
 '20100329',
 '20100330',
 '20100331',
 '20100401',
 '20100402',
 '20100405',
 '20100406',
 '20100407',
 '20100408',
 '20100409',
 '20100412',
 '20100413',
 '20100414',
 '20100415',
 '20100416',
 '20100419',
 '20100420',
 '20100421',
 '20100422',

In [25]:
len(lv2_index_rename)

2466

In [26]:
# Data Guide에서 받은 자료에 있는 모든 기업 & 기업코드를 토대로 만든 symbol_dic을 이용하여 lv2_companies에 있는 기업들의 기업코드를 symbol에 저장
symbol = []
for corp in lv2_companies:
    symbol.append(symbol_dic[corp])
symbol

['006840',
 '027410',
 '282330',
 '138930',
 '001040',
 '079160',
 '000120',
 '097950',
 '005830',
 '000990',
 '139130',
 '114090',
 '078930',
 '006360',
 '007070',
 '012630',
 '294870',
 '082740',
 '001060',
 '096760',
 '105560',
 '002380',
 '009440',
 '001940',
 '025000',
 '030200',
 '033780',
 '093050',
 '003550',
 '034220',
 '001120',
 '068870',
 '051900',
 '032640',
 '011070',
 '066570',
 '108670',
 '051910',
 '079550',
 '006260',
 '010120',
 '035420',
 '005940',
 '010060',
 '005490',
 '100840',
 '064960',
 '003570',
 '010950',
 '034120',
 '034730',
 '011790',
 '001740',
 '006120',
 '096770',
 '285130',
 '017670',
 '000660',
 '005610',
 '011810',
 '077970',
 '067250',
 '071970',
 '084870',
 '035250',
 '000050',
 '010130',
 '002240',
 '009290',
 '017900',
 '007690',
 '002990',
 '011780',
 '001210',
 '073240',
 '000270',
 '024110',
 '003920',
 '025860',
 '002350',
 '251270',
 '006280',
 '005250',
 '004370',
 '019680',
 '005270',
 '004130',
 '008060',
 '000210',
 '001680',
 '047040',

In [27]:
len(symbol)

310

In [28]:
def get_closing_price(code):  # 개별기업코드 입력 시 종가 반환
    
    import FinanceDataReader as fdr
   
    df = fdr.DataReader(code, start = '2010-01-01', end = '2019-12-31')
    df = df['Close']
    
    return df

In [29]:
def get_return(code):  # 개별기업코드 입력 시 수익률 반환
    
    import FinanceDataReader as fdr
    
    df = fdr.DataReader(code, start = '2010-01-01', end = '2019-12-31')
    df = df['Change']
    
    return df

In [32]:
## 필요 없음
def get_monthly_price(code):  # 개별기업코드 입력 시 월별 종가 반환
    
    from pykrx import stock
    
    df = stock.get_market_ohlcv_by_date("20100101", "20191231", code, "m")
    df = df['종가']
    
    return df

새로운 방법 : concat하기 → columns의 이름을 바꿔줘야 함

In [31]:
# '알루코' 삭제 → 주가 데이터 없음
del symbol[178]
del lv2_companies[178]
del lv2_companies_rename[178]

In [114]:
price_df = pd.DataFrame()
for process,x in enumerate(symbol):
    print("Process : {} | Total : {}".format(process+1,len(symbol)), end='\r')
    char_price_df = get_closing_price(x)
    price_df = pd.concat([price_df, char_price_df], axis=1)

In [33]:
## 필요 없음
monthly_price_df = pd.DataFrame()
for process,x in enumerate(symbol):
    print("Process : {} | Total : {}".format(process+1,len(symbol)), end='\r')
    char_monthly_price_df = get_monthly_price(x)
    monthly_price_df = pd.concat([monthly_price_df, char_monthly_price_df], axis=1)

In [34]:
## 필요 없음
monthly_price_df

,종가,종가,종가,종가,종가,종가,종가,종가,종가,종가,...,종가,종가,종가,종가,종가,종가,종가,종가,종가,종가
2010-01-31,9955,NaN,NaN,NaN,63367,21500,53800,209000,33550,6080,...,17750,NaN,19850,4100,48113,NaN,3775,NaN,NaN,16859
2010-02-28,9580,NaN,NaN,NaN,64876,22900,56100,209500,31500,6250,...,18900,NaN,20000,4960,51782,NaN,3932,NaN,NaN,16829
2010-03-31,11145,NaN,NaN,NaN,63556,23800,66100,225000,32300,7370,...,18300,NaN,22700,6680,48053,NaN,3790,NaN,NaN,16813
2010-04-30,11364,NaN,NaN,NaN,60821,23300,62400,222000,35300,8420,...,20650,NaN,22450,7650,46811,NaN,3618,NaN,NaN,16531
2010-05-31,11364,NaN,NaN,NaN,57144,24150,55500,214000,33100,10600,...,22550,NaN,21200,11450,40183,NaN,3153,NaN,NaN,15028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,33300,5750.0,203500.0,6700.0,81500,33550,143000,228500,47900,14700,...,23900,93100.0,78800,2825,86400,35400.0,9100,57200.0,6310.0,21300
2019-09-30,31450,6000.0,197000.0,7190.0,82000,33400,144500,234500,51600,16000,...,26400,91500.0,86200,3450,83800,31450.0,8480,57900.0,6170.0,21400
2019-10-31,31300,5750.0,178000.0,6950.0,82400,33750,157500,228500,50500,17000,...,25250,84500.0,77500,3650,82700,28300.0,8440,57600.0,5800.0,22100
2019-11-30,31500,5570.0,168000.0,6960.0,92100,36750,157500,248000,55700,19000,...,28000,85000.0,84700,3555,79400,25350.0,7830,51600.0,6010.0,21500


In [109]:
stock.get_market_ticker_name('021570')

'알루코'

In [110]:
fdr.DataReader('021570', start = '2010-01-01', end = '2019-12-31')

""


In [115]:
price_df

,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,...,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close
2010-01-04,10331,NaN,NaN,NaN,58652,21100,54500,207000,34200,7500,...,19450,NaN,21950,3915,50480,NaN,3726,NaN,NaN,17885
2010-01-05,10206,NaN,NaN,NaN,59595,21200,54400,220000,33600,7550,...,19300,NaN,22300,3785,52255,NaN,3702,NaN,NaN,17312
2010-01-06,10206,NaN,NaN,NaN,59595,21600,56400,220500,35450,7410,...,20200,NaN,22200,3910,54859,NaN,3755,NaN,NaN,17613
2010-01-07,10300,NaN,NaN,NaN,60444,21750,56600,218000,35650,7460,...,20200,NaN,21550,3865,53143,NaN,3785,NaN,NaN,17312
2010-01-08,10362,NaN,NaN,NaN,63556,22800,57600,218000,35300,7510,...,20000,NaN,21350,3975,55037,NaN,3804,NaN,NaN,17432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,33500,5570.0,165500.0,7870.0,96500,34900,151500,243500,54600,25950,...,27900,81000.0,86300,3145,82600,26400.0,7910,53900.0,6080.0,22450
2019-12-24,34500,5530.0,165500.0,7850.0,96000,34100,150000,242000,54500,26650,...,27900,81000.0,84000,3270,83800,26150.0,8020,54000.0,6120.0,22650
2019-12-26,34900,5580.0,165500.0,7770.0,97000,33800,150000,243000,54500,26650,...,27950,81000.0,83800,3250,84300,26200.0,7970,52600.0,6250.0,22450
2019-12-27,34850,5600.0,168500.0,7620.0,96400,33950,154500,250000,52800,28150,...,27150,80200.0,87300,3270,79600,26300.0,8110,53800.0,6090.0,21300


In [117]:
price_df.columns = lv2_companies_rename
price_df.columns

Index(['AK홀딩스', 'BGF', 'BGF리테일', 'BNK금융지주', 'CJ', 'CJ CGV', 'CJ대한통운', 'CJ제일제당',
       'DB손해보험', 'DB하이텍',
       ...
       '현대해상', '현대홈쇼핑', '호텔신라', '화신', '효성', '효성중공업', '후성', '휠라홀딩스', '휴비스',
       '휴켐스'],
      dtype='object', length=309)

In [128]:
price_df.index = lv2_index_rename
price_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
20100104,10331,NaN,NaN,NaN,58652,21100,54500,207000,34200,7500,...,19450,NaN,21950,3915,50480,NaN,3726,NaN,NaN,17885
20100105,10206,NaN,NaN,NaN,59595,21200,54400,220000,33600,7550,...,19300,NaN,22300,3785,52255,NaN,3702,NaN,NaN,17312
20100106,10206,NaN,NaN,NaN,59595,21600,56400,220500,35450,7410,...,20200,NaN,22200,3910,54859,NaN,3755,NaN,NaN,17613
20100107,10300,NaN,NaN,NaN,60444,21750,56600,218000,35650,7460,...,20200,NaN,21550,3865,53143,NaN,3785,NaN,NaN,17312
20100108,10362,NaN,NaN,NaN,63556,22800,57600,218000,35300,7510,...,20000,NaN,21350,3975,55037,NaN,3804,NaN,NaN,17432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191223,33500,5570.0,165500.0,7870.0,96500,34900,151500,243500,54600,25950,...,27900,81000.0,86300,3145,82600,26400.0,7910,53900.0,6080.0,22450
20191224,34500,5530.0,165500.0,7850.0,96000,34100,150000,242000,54500,26650,...,27900,81000.0,84000,3270,83800,26150.0,8020,54000.0,6120.0,22650
20191226,34900,5580.0,165500.0,7770.0,97000,33800,150000,243000,54500,26650,...,27950,81000.0,83800,3250,84300,26200.0,7970,52600.0,6250.0,22450
20191227,34850,5600.0,168500.0,7620.0,96400,33950,154500,250000,52800,28150,...,27150,80200.0,87300,3270,79600,26300.0,8110,53800.0,6090.0,21300


In [129]:
price_df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\price_df.csv', encoding='euc-kr')

같은 방법으로 수익률 데이터 가져오기

In [120]:
return_df = pd.DataFrame()
for process,x in enumerate(symbol):
    print("Process : {} | Total : {}".format(process+1,len(symbol)), end='\r')
    char_return_df = get_return(x)
    return_df = pd.concat([return_df, char_return_df], axis=1)

In [121]:
return_df

,Change,Change,Change,Change,Change,Change,Change,Change,Change,Change,...,Change,Change,Change,Change,Change,Change,Change,Change,Change,Change
2010-01-04,-0.002992,NaN,NaN,NaN,-0.007983,-0.040909,-0.038801,0.007299,0.055556,0.060820,...,0.040107,NaN,0.065534,0.018205,0.001170,NaN,-0.006665,NaN,NaN,-0.024650
2010-01-05,-0.012100,NaN,NaN,NaN,0.016078,0.004739,-0.001835,0.062802,-0.017544,0.006667,...,-0.007712,NaN,0.015945,-0.033206,0.035162,NaN,-0.006441,NaN,NaN,-0.032038
2010-01-06,0.000000,NaN,NaN,NaN,0.000000,0.018868,0.036765,0.002273,0.055060,-0.018543,...,0.046632,NaN,-0.004484,0.033025,0.049833,NaN,0.014317,NaN,NaN,0.017387
2010-01-07,0.009210,NaN,NaN,NaN,0.014246,0.006944,0.003546,-0.011338,0.005642,0.006748,...,0.000000,NaN,-0.029279,-0.011509,-0.031280,NaN,0.007989,NaN,NaN,-0.017090
2010-01-08,0.006019,NaN,NaN,NaN,0.051486,0.048276,0.017668,0.000000,-0.009818,0.006702,...,-0.009901,NaN,-0.009281,0.028461,0.035640,NaN,0.005020,NaN,NaN,0.006932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,0.007519,0.012727,0.015337,-0.002535,0.002077,-0.002857,-0.013029,0.008282,-0.003650,-0.022599,...,0.005405,0.003717,0.002323,0.006400,0.012255,0.011494,-0.027060,0.022770,0.001647,0.004474
2019-12-24,0.029851,-0.007181,0.000000,-0.002541,-0.005181,-0.022923,-0.009901,-0.006160,-0.001832,0.026975,...,0.000000,0.000000,-0.026651,0.039746,0.014528,-0.009470,0.013906,0.001855,0.006579,0.008909
2019-12-26,0.011594,0.009042,0.000000,-0.010191,0.010417,-0.008798,0.000000,0.004132,0.000000,0.000000,...,0.001792,0.000000,-0.002381,-0.006116,0.005967,0.001912,-0.006234,-0.025926,0.021242,-0.008830
2019-12-27,-0.001433,0.003584,0.018127,-0.019305,-0.006186,0.004438,0.030000,0.028807,-0.031193,0.056285,...,-0.028623,-0.009877,0.041766,0.006154,-0.055753,0.003817,0.017566,0.022814,-0.025600,-0.051225


In [122]:
return_df.columns = lv2_companies_rename
return_df.columns

Index(['AK홀딩스', 'BGF', 'BGF리테일', 'BNK금융지주', 'CJ', 'CJ CGV', 'CJ대한통운', 'CJ제일제당',
       'DB손해보험', 'DB하이텍',
       ...
       '현대해상', '현대홈쇼핑', '호텔신라', '화신', '효성', '효성중공업', '후성', '휠라홀딩스', '휴비스',
       '휴켐스'],
      dtype='object', length=309)

In [130]:
return_df.index = lv2_index_rename
return_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
20100104,-0.002992,NaN,NaN,NaN,-0.007983,-0.040909,-0.038801,0.007299,0.055556,0.060820,...,0.040107,NaN,0.065534,0.018205,0.001170,NaN,-0.006665,NaN,NaN,-0.024650
20100105,-0.012100,NaN,NaN,NaN,0.016078,0.004739,-0.001835,0.062802,-0.017544,0.006667,...,-0.007712,NaN,0.015945,-0.033206,0.035162,NaN,-0.006441,NaN,NaN,-0.032038
20100106,0.000000,NaN,NaN,NaN,0.000000,0.018868,0.036765,0.002273,0.055060,-0.018543,...,0.046632,NaN,-0.004484,0.033025,0.049833,NaN,0.014317,NaN,NaN,0.017387
20100107,0.009210,NaN,NaN,NaN,0.014246,0.006944,0.003546,-0.011338,0.005642,0.006748,...,0.000000,NaN,-0.029279,-0.011509,-0.031280,NaN,0.007989,NaN,NaN,-0.017090
20100108,0.006019,NaN,NaN,NaN,0.051486,0.048276,0.017668,0.000000,-0.009818,0.006702,...,-0.009901,NaN,-0.009281,0.028461,0.035640,NaN,0.005020,NaN,NaN,0.006932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191223,0.007519,0.012727,0.015337,-0.002535,0.002077,-0.002857,-0.013029,0.008282,-0.003650,-0.022599,...,0.005405,0.003717,0.002323,0.006400,0.012255,0.011494,-0.027060,0.022770,0.001647,0.004474
20191224,0.029851,-0.007181,0.000000,-0.002541,-0.005181,-0.022923,-0.009901,-0.006160,-0.001832,0.026975,...,0.000000,0.000000,-0.026651,0.039746,0.014528,-0.009470,0.013906,0.001855,0.006579,0.008909
20191226,0.011594,0.009042,0.000000,-0.010191,0.010417,-0.008798,0.000000,0.004132,0.000000,0.000000,...,0.001792,0.000000,-0.002381,-0.006116,0.005967,0.001912,-0.006234,-0.025926,0.021242,-0.008830
20191227,-0.001433,0.003584,0.018127,-0.019305,-0.006186,0.004438,0.030000,0.028807,-0.031193,0.056285,...,-0.028623,-0.009877,0.041766,0.006154,-0.055753,0.003817,0.017566,0.022814,-0.025600,-0.051225


In [131]:
return_df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\return_df.csv', encoding='euc-kr')